In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ee
from google.oauth2 import service_account

# Initialize Earth Engine API
service_account_key = r'C:\Users\serafino\Desktop\Water Management System\ee-eugenioserafino01-dca7dda5348b.json'

# Define the required OAuth scope for Earth Engine
scopes = ['https://www.googleapis.com/auth/earthengine']

# Create credentials with the specified scope
credentials = service_account.Credentials.from_service_account_file(
    service_account_key, scopes=scopes
)

# Initialize Earth Engine with the credentials
ee.Initialize(credentials=credentials)

# Define location, date range, and area of interest
start_date = '2020-01-01'
end_date = '2023-12-31'

# Define a polygon with coordinates directly as a list of lists
pontelongo_coordinates = [
    [12.021280, 45.251061],
    [12.021719, 45.250949],
    [12.022229, 45.251746],
    [12.022891, 45.251549],
    [12.023380, 45.252315],
    [12.023846, 45.252183],
    [12.024085, 45.252617],
    [12.022100, 45.252718],
    [12.021280, 45.251061]  # Closing the polygon by repeating the first point
]

# Create an Earth Engine Geometry object
location_polygon = ee.Geometry.Polygon([pontelongo_coordinates])

# The rest of your code remains the same


# Load Sentinel-2 Harmonized image collection
collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
               .filterDate(start_date, end_date) \
               .filterBounds(location_polygon)

# Function to calculate indices
def calculate_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndmi = image.normalizedDifference(['B8', 'B11']).rename('NDMI')
    swir = image.select(['B11']).rename('SWIR')
    return image.addBands([ndvi, ndmi, swir])

# Map the function to the collection
collection_with_indices = collection.map(calculate_indices)

# Reduce by region to get mean, min, and max values for each day
def get_stats_indices(image):
    stats_dict = image.reduceRegion(
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.minMax(),
            sharedInputs=True
        ),
        geometry=location_polygon,
        scale=10,
        maxPixels=1e9
    )
    stats_dict = stats_dict.set('date', image.date().format('YYYY-MM-dd'))
    return ee.Feature(None, stats_dict)

# Apply to each image and convert to a FeatureCollection
stats_indices_fc = ee.FeatureCollection(collection_with_indices.map(get_stats_indices))

# Convert to a list and fetch data
indices_data = stats_indices_fc.getInfo()['features']

# Parse data into a DataFrame
dates = []
ndvi_mean, ndvi_min, ndvi_max = [], [], []
ndmi_mean, ndmi_min, ndmi_max = [], [], []
swir_mean, swir_min, swir_max = [], [], []

for feature in indices_data:
    properties = feature['properties']
    dates.append(properties['date'])
    
    # NDVI stats
    ndvi_mean.append(properties.get('NDVI_mean', np.nan))
    ndvi_min.append(properties.get('NDVI_min', np.nan))
    ndvi_max.append(properties.get('NDVI_max', np.nan))
    
    # NDMI stats
    ndmi_mean.append(properties.get('NDMI_mean', np.nan))
    ndmi_min.append(properties.get('NDMI_min', np.nan))
    ndmi_max.append(properties.get('NDMI_max', np.nan))
    
    # SWIR stats
    swir_mean.append(properties.get('SWIR_mean', np.nan))
    swir_min.append(properties.get('SWIR_min', np.nan))
    swir_max.append(properties.get('SWIR_max', np.nan))

# Create DataFrame
df = pd.DataFrame({
    'date': pd.to_datetime(dates),
    'NDVI_mean': ndvi_mean, 'NDVI_min': ndvi_min, 'NDVI_max': ndvi_max,
    'NDMI_mean': ndmi_mean, 'NDMI_min': ndmi_min, 'NDMI_max': ndmi_max,
    'SWIR_mean': swir_mean, 'SWIR_min': swir_min, 'SWIR_max': swir_max
})

# Handle any missing values (e.g., by interpolation)
df = df.sort_values(by='date')
df = df.interpolate(method='linear')

# Print the DataFrame
print(df)



          date  NDVI_mean  NDVI_min  NDVI_max  NDMI_mean  NDMI_min  NDMI_max  \
0   2020-01-04   0.228241 -0.015783  0.428803   0.050861 -0.183265  0.242532   
1   2020-01-09   0.048872 -0.017316  0.179046  -0.055524 -0.126995  0.040330   
2   2020-01-14   0.034191 -0.015829  0.074820   0.037386 -0.017925  0.092674   
3   2020-01-19   0.031914  0.007508  0.076923   0.287413  0.242857  0.390452   
4   2020-01-24   0.033319 -0.014739  0.221686   0.076023 -0.078397  0.181905   
..         ...        ...       ...       ...        ...       ...       ...   
283 2023-12-09   0.074217  0.041865  0.182525  -0.042913 -0.105137  0.045862   
284 2023-12-14   0.025127 -0.006118  0.088210   0.043528  0.008648  0.080933   
285 2023-12-19   0.042084 -0.127820  0.543958  -0.017482 -0.224472  0.343816   
286 2023-12-24   0.060759  0.043841  0.087969  -0.006509 -0.101423  0.060324   
287 2023-12-29  -0.004662 -0.059650  0.041508   0.140377  0.004375  0.228738   

       SWIR_mean  SWIR_min  SWIR_max  
